# Calligaro Nicolas
## Prédiction et Classification
Readme :\
Ce jupyter se concentre sur la production de jeu de donnée afin d'appliquer des algos de prédiction et de classement.

In [1]:
import pandas as pd
import pickle
import numpy as np
import ipywidgets as iws
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import r2_score
df_clas=pickle.Unpickler(open("df_clas.pkl",'rb')).load()
df_tech=pickle.Unpickler(open("df_tech.pkl",'rb')).load()
df_clas.shape,df_tech.shape

((4142, 23), (33, 22))

# Prédiction par skipper
### On veut répondre à la question :
#### Quel classement aura ce skipper au vu de ses performances et de celle des autres pour le dernier créneau horaire.

In [16]:
#Colonne sur lesquels on va travailler
df_clean= df_clas.drop(df_clas.columns[[1,3,4,6,7,8,12,16,22]],axis=1)

In [5]:
choice = iws.Dropdown(
    options=list(df_tech.Skipper),
    value=list(df_tech.Skipper)[0],
    description='Choix du Skipper:',
    disabled=False,
)
display(choice)

Dropdown(description='Choix du Skipper:', options=('jean le cam', 'armel tripon', 'sebastien simon', 'nicolas …

In [6]:
#Création des données :
#Pour chaque créneau horaire. on prend les infos du skipper + la moyenne des infos des autres skippers
#info : Vitess,VMG,Distance sur le créneau + Vitesse,VMG,Distance sur 24h
#On créé une ligne par créneau horaire.
skipper = df_tech[df_tech.Skipper == choice.value].index[0]

df_group = df_clean.groupby('Date full')

#On choisit un skipper

df_sol = pd.DataFrame(columns=[])

for i in df_group.groups.keys():
    df_tmp = df_group.get_group(i)
    df_cst =list(df_tmp[df_tmp['Num_voile']==skipper].iloc [:,3:].mean())
    df_cst.extend(list(df_tmp[df_tmp['Num_voile']!=skipper].iloc [:,3:].mean()))
    df_cst.extend(list(df_tmp[df_tmp['Num_voile']==skipper]['Rang']))
    df_sol = df_sol.append([df_cst])

df_sol.shape

(128, 23)

In [7]:
#Création du jeu d'entrainement
#On prends toutes les lignes créneaux temporel skipper + moyenne des autres 
#sauf la dernière (qui représente le dernier créneau horraire)
#Création du jeu de test
#On prend la dernière ligne 

dfX = df_sol[df_sol.columns[:-1]] #Toutes les colonnes sauf la dernière
dfy = df_sol[df_sol.columns[-1]]  #LA dernière colone qui représente le classement
X_train= dfX.iloc[:-1,]           #toutes les lignes sauf la dernière
y_train= dfy.iloc[:-1,]           #
X_test = dfX.iloc[-1:,]           #LA dernière ligne qui représente le dernier créneau horraire
y_test = dfy.iloc[-1:,]           #

reg_model = LinearRegression().fit(X_train,y_train)
df_sol.shape, dfX.shape,X_train.shape,X_test.shape,y_train.shape,y_test.shape

((128, 23), (128, 22), (127, 22), (1, 22), (127,), (1,))

In [8]:
reg_model = LinearRegression().fit(X_train,y_train)
test= list(np.floor(reg_model.predict(X_train)))
test[:5],list(y_train)[:5]
r2 = r2_score(list(y_train),test)

In [9]:
#On prédis le classement du skipper en fct de ces résultats et de celles des autres. on compare avec le classement effectif
prediction = list(np.floor(reg_model.predict(X_test)))
print(f"prédiction : {int(prediction[0])}, réalité : {y_test[0]}, Taux de fiabilité : {np.around(r2*100)}%")

prédiction : 1, réalité : 3, Taux de fiabilité : 74.0%


#### Notre prédicteur à un taux de fiabilité de 74% (ce qui est franchement mauvais...) il faudrait faire une étude plus poussé sur les variables retenus ou en produire d'autre.

# Classification
>Y = Spoil or not Spoil\
>X = Vitesse, VMG,Distance, classement du skipper
### On veux répondre à la questions: 
#### Est ce que ce skipper a un foils sur son bateau au vu de ses performances ?

In [17]:
#On retire les colonnes inutiles pour notre cas.
df_clean= df_clas.drop(df_clas.columns[[1,3,4,6,7,8,12,16,22]],axis=1)
#On rajoute la colonne foils pour la classification
df_upgrade = pd.DataFrame.join (df_clean,df_tech['foils']
                 ,on='Num_voile'
                )

In [18]:
#Colones étudiés :
df_upgrade = df_upgrade.iloc [:,[0,1,3,4,5,6,7,8,9,11,13,14]]#18 = Foils

In [13]:
#Création du jeu de donnée. on prend tout le DF (créneau horraire - skipper à l'étude)
df_group = df_upgrade.groupby('Num_voile')

skipper = 56

df_sol = pd.DataFrame(columns=[])

for i in df_group.groups.keys():
    if i == skipper:#On va créer le Xytest ici
        df_tmp = df_group.get_group(i)
        Xytest = df_tmp.drop(df_tmp.columns[[1,2]],axis=1)
    df_tmp = df_group.get_group(i)
    df_cst = df_tmp.drop(df_tmp.columns[[1,2]],axis=1)
    df_sol = df_sol.append([df_cst])

df_sol.shape,Xytest.shape

((4142, 10), (128, 10))

In [14]:
#création du jeu d'entrainement :
#toutes les données sauf celle concernant le skipper en question
#Création du jeu de test :
#Toutes les données du skipper en question (on ferra une moyenne de ces données)

X_train = df_sol[df_sol.columns[:-1]] #Toutes les colonnes sauf la dernière
y_train = df_sol[df_sol.columns[-1]]  #LA dernière colone

X_test = Xytest[df_sol.columns[:-1]]  #LA dernière ligne
y_test = Xytest[df_sol.columns[-1]]   #LA dernière colone

clas_model = SGDClassifier(max_iter=1000, tol=1e-3).fit(X_train,y_train)

#model = LinearRegression().fit(X_train,y_train)
df_sol.shape, X_train.shape,y_train.shape,Xytest.shape,X_test.shape,y_test.shape

((4142, 10), (4142, 9), (4142,), (128, 10), (128, 9), (128,))

In [15]:
#On définit si le bateau du skipper au vu de ses résultats est équipé de foils ou non
prediction = clas_model.predict(X_test)
j=0
for i in prediction:
    if i :
        j+1
print (f"prédiction {j>len(prediction)}, Réalité : {list(y_test)[-1]}")

prédiction False, Réalité : True
